# Erstellung des Korpus

In [1]:
import glob
import numpy as np
import os
import pandas as pd
import re

In [2]:
# Metadaten einlesen

md = pd.read_csv('metadata.csv', sep=';', encoding='utf-8')

In [3]:
files = []

for file in os.listdir('Texte'):
    files.append(file)

In [4]:
# Texte einlesen 

data = []

for file in os.listdir('Texte'):
    with open(os.path.join('Texte', file),'r', encoding='utf-8', errors='ignore') as myfile:
        data.append(myfile.read())

In [5]:
# Metadaten und Texte zusammenfügen (matcht den Dokumentnamen)

corpus = pd.DataFrame({'file': files, 'text': data})

df = pd.merge(md, corpus, how='left', left_on='doc', right_on='file')

In [6]:
# Wörter zählen und hinzfügen

df['words'] = df['text'].str.count(r'\w+')

In [7]:
# Spalten neu anordnen und unnötige entfernen

df = df[['doc', 'source', 'author', 'title', 'year', 'period', 'text type', 'text', 'words']]

In [8]:
df.head()

,doc,source,author,title,year,period,text type,text,words
0,diacorisCampanile2.txt,DiaCoris,Achille Campanile,Gli asparagi e l'immortalità dell'anima,1974,1951-1975,espositivo,"nonché interessarsi , non riesce addirittura a...",165.0
1,LISCampanile1.txt,LIS,Achille Campanile,Chi non fa rirecitare non s'aspetti di ritrasm...,1960,1951-1975,stampa,TV e attori s ' accapigliano su una cosa che n...,1179.0
2,LISCampanile2.txt,LIS,Achille Campanile,Belle prodezze/ Perry/ mettere nel sacco lo st...,1960,1951-1975,stampa,"Sempre novità , alla TV , sempre idee fresche ...",1188.0
3,LISCampanile3.txt,LIS,Achille Campanile,Peggio del monopolio il monopolio del monopolio,1960,1951-1975,stampa,"Nella severa aula della Consulta , davanti all...",1579.0
4,LISCampanile4.txt,LIS,Achille Campanile,Il triplice maledetto destino die refusi origi...,1960,1951-1975,stampa,Voi dite che questa grande invenzione che è la...,1214.0


In [9]:
df.shape 

(1575, 9)

In [10]:
# KWIC-Listen 
# (vorher einzeln in einheitliche Form gebracht, siehe Vorbereitungscode in den Ordnern der jeweiligen Korpora)

directory = 'KWIC'
kwic_files = glob.glob(os.path.join(directory, '*.csv'))

In [11]:
# alle KWIC-Listen einlesen 

frames = []

for kwic_file in kwic_files:
    df_new = pd.read_csv(kwic_file, sep=',', encoding='utf-8')
    frames.append(df_new)

In [12]:
# zu einem Dataframe zusammenfügen 

df_big = pd.concat(frames)

In [13]:
df_big.head()

,doc,source,author,title,year,period,text type,text,words
0,ita_news_2020_1M-sentences.txt,Corpora Italian,https://www.panorama.it/video/notizie/catanzar...,https://www.panorama.it/video/notizie/catanzar...,1970.0,1951-1975,stampa,È però arrivato il momento di tirarsi su di mo...,10
1,ita_news_2020_1M-sentences.txt,Corpora Italian,https://www.ilpaesenuovo.it/2020/12/10/lorosco...,https://www.ilpaesenuovo.it/2020/12/10/lorosco...,1970.0,1951-1975,stampa,Come altro aspetto da considerare è il fattore...,23
2,ita_news_2020_1M-sentences.txt,Corpora Italian,https://www.lanazione.it/pisa/cronaca/al-fisic...,https://www.lanazione.it/pisa/cronaca/al-fisic...,2000.0,1976-2000,stampa,È però “assolutamente cruciale che si tenga la...,25
3,ita_news_2020_1M-sentences.txt,Corpora Italian,https://www.smartworld.it/internet/costo-dei-v...,https://www.smartworld.it/internet/costo-dei-v...,2002.0,2001-2010,stampa,"E però, c'è la firma di Mediobanca anche sul m...",22
4,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://www.corriere.it/Primo_Piano/Spettacoli/...,http://www.corriere.it/Primo_Piano/Spettacoli/...,2005.0,2001-2010,stampa,"E cos ieri, parlando a Bari, il leader ha pron...",22


In [14]:
df_big.shape

(6269370, 9)

In [15]:
# KWIC-Listen und selbst erstelltes Korpus zu einem zusammenfügen

df_bigger = pd.concat([df_big, df], axis=0, join='outer', ignore_index=False)

In [16]:
df_bigger.head()

,doc,source,author,title,year,period,text type,text,words
0,ita_news_2020_1M-sentences.txt,Corpora Italian,https://www.panorama.it/video/notizie/catanzar...,https://www.panorama.it/video/notizie/catanzar...,1970.0,1951-1975,stampa,È però arrivato il momento di tirarsi su di mo...,10.0
1,ita_news_2020_1M-sentences.txt,Corpora Italian,https://www.ilpaesenuovo.it/2020/12/10/lorosco...,https://www.ilpaesenuovo.it/2020/12/10/lorosco...,1970.0,1951-1975,stampa,Come altro aspetto da considerare è il fattore...,23.0
2,ita_news_2020_1M-sentences.txt,Corpora Italian,https://www.lanazione.it/pisa/cronaca/al-fisic...,https://www.lanazione.it/pisa/cronaca/al-fisic...,2000.0,1976-2000,stampa,È però “assolutamente cruciale che si tenga la...,25.0
3,ita_news_2020_1M-sentences.txt,Corpora Italian,https://www.smartworld.it/internet/costo-dei-v...,https://www.smartworld.it/internet/costo-dei-v...,2002.0,2001-2010,stampa,"E però, c'è la firma di Mediobanca anche sul m...",22.0
4,ita_news_2005-2009_1M-sentences.txt,Corpora Italian,http://www.corriere.it/Primo_Piano/Spettacoli/...,http://www.corriere.it/Primo_Piano/Spettacoli/...,2005.0,2001-2010,stampa,"E cos ieri, parlando a Bari, il leader ha pron...",22.0


In [17]:
df_bigger.shape

(6270945, 9)

In [18]:
# Text von Tags befreien

def parser(text):
    return re.sub('<[^<]+?>', '', str(text))

df_bigger['text'] = df_bigger.text.apply(parser)

In [19]:
# NaN durch 0 ersetzen (nur für den Fall, um spätere Fehler zu vermeiden)

df_bigger = df_bigger.fillna(0)

In [20]:
# chronologisch ordnen

df_bigger = df_bigger.sort_values(by=['year'])

In [21]:
df_bigger

,doc,source,author,title,year,period,text type,text,words
1042,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0
854,Poesia.IV.1.Testo.txt,MIDIA,Giuseppe Paolucci (Alessi Cillenio),Poesie,1700.0,1700-1750,poesia,IV. 1. Rime degli Arcadi: Alessi Cillenio (Giu...,10862.0
1506,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0
967,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0
424,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0
...,...,...,...,...,...,...,...,...,...
86167,JSIorrendamente.csv,JSI,cronachesalerno.it,cronachesalerno.it,2021.0,2017-2021,web,ultimo è stato molto complicato e la pandemia ...,40.0
86168,JSIorrendamente.csv,JSI,95047.it,95047.it,2021.0,2017-2021,web,esclusivamente in modalità a distanza. Resta ...,34.0
86169,JSIorrendamente.csv,JSI,gds.it,gds.it,2021.0,2017-2021,web,che l''Indice Fao dei prezzi dei prodotti alim...,37.0
86162,JSIorrendamente.csv,JSI,zazoom.it,zazoom.it,2021.0,2017-2021,web,câ€™Ã¨ un problema. Limiti della Regione nell...,39.0


In [22]:
# speichern

df_bigger.to_csv('Korpus/corpus_complete.csv', encoding='utf-8', index=False)